# Convolutional Neural Networks and Feature Hierarchies

## An Overview of CNNs

Extracting salient features is vital to an ML algorithm. CNNs automatically learn the features from raw data that are most useful. CNN layers thus act as a feature extraction mechanism, where early layers extract low level features and later layers (often fully connected) combine these features into higher level ones with more complexity to perform a prediction.

CNNs are excellent at image processing and are thus widely associated with it. CNNs take in an input image, consider local patches of pixels (local receptive field), and combine them with a kernel. This has two main effects-
1) Sparse connectivity: Since each element of the feature map only corresponds to a smaller local patch of pixels, early layers are sparesly connected.
2) Parameter sharing: Same weights/kernels get used over different patches of the input image.

This has a huge reduction on the number of weights necessary, and typically results in an improved ability to capture relevant features. This works well in image recognition tasks since nearby pixels are typically more relevant than distant ones, since changes (or the lack thereof) in local pixel values typically correspond to contours, object separation, etc.

CNNs typically have several convolutional and subsampling (or pooling) layers, forllowed by one or two fully connected layers at the end. The pooling layers do not have any learnable paramters, but all other layers do. Pooling layers ahve the effect of coarse graining an image.

## Convolutions and Related Parameters

A discrete convolution, the building block of a CNN, is a mathematical operation between two tensors.

### One Dimension

In one dimension, a convolution is defined between two tensors in a basis dependent manner:
$$y_i = (x \ast w)_i = \sum_{k = -\infty}^{\infty} x_{i - k} w_k$$
Given the convolution operator $C$ (with appropriately placed 1s and 0s), this can be written as $y = \langle x, C w \rangle$.

When we work with finite dimensional vectors, typically their vector space is considered embedded in a higher dimensional subspace, where their projections into the additional dimensions is 0. This is called padding/zero-padding. Padding parameter $p$ adds $2 p$ dimensions. In practice, this adds $p$ zeros on either end of the vector (and appropriately extends the operator $C$). Then, this sum goes from $0$ to $dim(x) + 2p$, where $dim$ here refers to the vector space that the vector belongs to. Note that $w$ and $x$ can have different dimension.

#### Cross-Correlation

Cross-correlation is closely realted to convolution. In fact, while a convolution "flips" the fliter (or the signal, depending on convention), correlation does not. More formally, if cross-correlation is denoted by $\star$, $$f(t) \star g(t) = f(-t) \ast g(t).$$ If the vectors are complex, this changes to $$f(t) \star g(t) = \bar{f}(-t) \ast g(t),$$ where the bar stands for the complex conjugation of $f$. In terms of finite dimensional vector spaces, we have $$y_i = (x \star w)_i = \sum_{k = -\infty}^{\infty} x_{i + k} w_k .$$ Most deep learning frameworks implement cross-correlation but typically refer to it as convolution, which is typical in the field.

#### Shift/Stride
The shift, or stride, denoted $s$, is another hyperparameter of a convolution. While a typical convolution "moves" the filter over by one step for the next element, the stride can change that to a larger number. In essence, a stride of 2 deletes every other element of a convolution with stride 1, while $s = 3$, for example, starts with element 0, skips 1, 2, computes element 3, skips 4, 5, etc. The stride 

#### Padding
The choice of padding dimension impacts the treatment of boundary terms. For example, in the case of $n = dim(x) = 5$, $m = dim(w) = 3$ and $p = 0$, we note that the first input to $x$, $x_0$ is only used once in the computation of the convolution (while computing $(x \ast w)_0$, while $x_1$ is is used in the computation of 2 different elements, $(x \ast w)_0$ and $(x \ast w)_1$, while $x_2$ has the greated impact, appearing in 3 elements of the convolution. If $p=2$, however, all elements are treated on the same footing, appearing in the computation of 3 elements of the convolution.

There are 3 main forms of padding:

1) Full: The padding dimension is set to $p = dim(w) - 1 = m - 1$. This isn't common in CNN architechtures, since it increases the dimensions of the convolution output.

2) Same: This padding sets $p$ such that the output size is the same as the size of the input $x$. In the case of $s= 1$. this works out to $p = (m - 1)/2$. In the case of $m$ odd, this typically warrants asymmetric padding, with $\mathrm{floor}((m - 1)/2)$ on one end and $\mathrm{ceil}((m - 1)/2)$ on the other. The same mode is used most commonly for CNNs.

3) Valid: This simply uses $p=0$. Typically, this reduction in size is not common, and the reduction of tensor volumes is done with pooling, since valid padding usually produces poor results.

#### Resulting Dimensions
After padding and adjusting the stride, the result of the convolution can have a dramatically different result than $dim(x) - dim(w) + 1$, the case of valid padding with a stride of $s = 1$. The resultant output size is: $$dim(x \ast w) = \lfloor \frac{dim(x) + 2p - dim(w)}{2 s} \rfloor + 1 = \lfloor \frac{n + 2p - m}{2s} \rfloor + 1$$

In [1]:
import numpy as np

def conv1d(x, w, p=0, s=1):
    w_flip = np.array(w[::-1])
    x_padded = np.array(x)
    
    if p:
        pad = np.zeros(shape=p)
        x_padded = np.concatenate([pad, x_padded, pad])
    
    result = []
    for i in range(0, int(len(x_padded) - len(w_flip)) + 1, s):
        result.append(np.sum(x_padded[i:i + w_flip.shape[0]] * w_flip))
    
    return np.array(result)

x = [1, 3, 543, 1, 32, 21, 2]
w = [1, 0, 4, -3, 1]

print(f'1D Convolution of {x} with {w} with p = 2 and s = 1 is {conv1d(x, w, p=2, s=1)}')
print(f'Results with NumPy: {np.convolve(np.array(x), np.array(w), mode="same")}')


1D Convolution of [1, 3, 543, 1, 32, 21, 2] with [1, 0, 4, -3, 1] with p = 2 and s = 1 is [  547.    10.  2196. -1601.   670.   -11.   -23.]
Results with NumPy: [  547    10  2196 -1601   670   -11   -23]


### Two Dimensions and Beyond
Much of the discussion pertaining to 2 dimensions remains pertinent and nearly identical in nature, just modified to 2 dimensions. In the case of 2 dimensions, the formula is: $$ Y_{ij} = (X \ast W)_{ij} = \sum_{k_1 = - \infty}^{\infty} \sum_{k_2 = - \infty}^{\infty} X_{i-k_1\, j-k_2} W_{k_1 k_2}.$$

The $i,j$ element of the convolution simplifies if the filter 2-tensor $W$ is separable, i.e. can be decomposted as $W = w_1 \otimes w_2$. Then, the 2D convolution is simply the convolution of each row of $X$ with $w_2$, and the convolution of each column of the result with $w_1$. This is possible with a Gaussian kernel or a Sobel X-gradient filter.

The padding and stride now become 2 dimensional hyperparameters, to indicate padding in each direction and stride in each direction. This extension also follows into higher dimensions.

In [2]:
from scipy.signal import convolve2d

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_flip = np.array(W)[::-1, ::-1]

    X_unpadded = np.array(X)
    X_padded = np.zeros(shape=(X_unpadded.shape[0] + 2 * p[0], X_unpadded.shape[1] + 2 * p[1]))

    X_padded[p[0]:p[0] + X_unpadded.shape[0], p[1]:p[1] + X_unpadded.shape[1]] = X_unpadded

    result = []

    for i in range(0, int((X_padded.shape[0] - W_flip.shape[0]) / s[0]) + 1, s[0]):
        result.append([])
        for j in range(0, int((X_padded.shape[1] - W_flip.shape[1]) / s[1]) + 1, s[1]):
            sub_conv = X_padded[i:i + W_flip.shape[0], j:j + W_flip.shape[1]]
            result[-1].append(np.sum(sub_conv * W_flip))
    
    return np.array(result)

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W =  [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print(f'2D Convolution of {X} with {W} with p = (2, 2) and s = (1, 1) is\n{conv2d(X, W, p=(1, 1), s=(1, 1))}\n')
print(f'Results with SciPy:\n{convolve2d(X, W, mode="same")}')

2D Convolution of [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]] with [[1, 0, 3], [1, 2, 1], [0, 1, 1]] with p = (2, 2) and s = (1, 1) is
[[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]

Results with SciPy:
[[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


### Subsampling / Pooling

A pooling is an operation that takes a neighbourhood of values and creates a single value out of them. Typically, this is done by breaking the original matrix down into (conventionally but not necessarily) separate windows of size $n_1 \times n_2$, denoted $P_{n_1 \times n_2}$ - so a $12 \times 12$ matrix pooled with $P_{n_1 \times n_2}$ is broken down into 16 different $3 \times 3$ blocks, and then an operation is performed on each of each of these blocks to output a $4 \times 4$ matrix. Pooling reduces the size of the feature space, boosting computational efficiency and potentially reducing overfitting issues. Typically, there are 2 types of pooling operations used in CNNs:

1) Max-pooling: For each block, only the maximum element is taken. This induces a local invariance - small changes in a neighborhood do not change the result of max-pooling, which generates features that are more robust to noise in the input data.

2) Average/Mean-pooling: The output from each block is the average of the elements in the block.

Note: To have non-overlapping pooling, the stride parameter has to match the pooling block sizes. This is typical in CNN implementations, but not always the case. If the stride parameter is smaller than, or not an integer multiple of the pooling, overlaps must occur.


### Implmentation of a CNN
For a regular NN, the linear operation is typically $Z = W \cdot X + b$. However, in the case of a CNN, the matrix multiplication is replaced by a convolution $Z = W \ast X + b$. Aside from pooling (which just adds another layer with no learning parameters associated), the rest remains the same: activation functions are still performed on the outputs $A = \sigma(Z)$. What is really being learned here are the convolution filters/kernels $W$.

While implementing a CNN for image recognition, we have to deal with 3 channels (RGB) or 4 (RGB + alpha, the transparancy). Each channel has a 2D matrix associated with it, for the set of pixels displaying the image. Thus, for $C_{in}$ channels in the input, each filter $W$ corresponds to a certain channel $W_c$. Then, the convolution is $$ Z = b + \sum_{c = 1}^{C_{in}} W_c \ast X_c.$$ Since we typically want multiple nodes in the output, $W_c$ typically has indices $W_{ij,c,c'}$ where the $c'$ index corresponds to the $c'^{th}$ node in the next layer (pre-activation).

## Preventing Overfitting

### Regularization

The capacity of a network refers to the complexity of a function that it can learn to approximate. While small networks tend to underfit, very large networks will tend to overfit. Since most applications don't have an a-priori prescription for the size of a neural network with respect to a particular application (unless the problem is simple enough to be solved with classical ML), it is typical to use a large NN. The same is true for CNNs. To prevent overfitting, one or more regularization schemes can be employed. It is possible to use $L_p$ regularization for these purposes, with $L_2$ regularization being the most common choice. This can simply be added to the loss function.

In [5]:
# Exmample of L2 loss added into the Loss fn

import torch.nn as nn
import torch

loss_fn = nn.BCELoss()
loss = loss_fn(torch.tensor([0.9]), torch.tensor([1.0]))

l2_lambda = 1e-3
conv_layer = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5)

l2_penalty = l2_lambda * sum([(p ** 2).sum() for p in conv_layer.parameters()])

loss_L2 = loss + l2_penalty

linear_layer = nn.Linear(10, 16)

l2_penalty = l2_lambda * sum([(p ** 2).sum() for p in conv_layer.parameters()])

loss_L2 - loss + l2_penalty

tensor(0.0035, grad_fn=<AddBackward0>)

### Dropout

Another popular way to prevent, or reduce the degree of, overfitting is droptout. This is usually applied to hidden units of later layers. At every training iteration/mini-batch, a fraction of the hidden units is dropped with the probability $p_{drop}$. This probability is a hyperparameter of training with a common choice of $p_{drop} = 0.5$. When performing this dropout on an iteration, the weights associated with the remaining neurons are rescaled durign evaluation, when all neurons are kept active (say halving the activations for active neurons). In practice, it is easier to scale the activations of inactive neurons by a factor of 2 during training - this is known as inverse dropout, and is comon among implementations, including PyTorch.

The rendomized dropout forces the network to learn redundant representations of the data - a particular set of hidden units that might otherwise learn features that are too specific to the training data would likely no longer be trained to do so. This makes the network learn more general features that are more robust to noise and training set particulars.

In effect, this is a consensus of an ensemble of models, done in a very efficient manner, sampling over $M = 2^h$ models for $h$ hidden units that face dropout. The main distinction is the shared weights over all models, which is a form of regularization to the mean. During prediction, we can average over the models via a geometric mean: $$p_{Ensemble} = \left( \prod_{j = 1}^{M}p^{\{i\}} \right )^{1/M}.$$

This is computationally expensive, but the geometric mean can be approximated by scaling the predictions of the final model sampled during training by a factor of $\frac{1}{1 - p}$. This is exact in the case of linear models.

## Loss Functions - Classification

If there are 2 classes, binary cross-entropy is used, with either class probabilities as inputs, or logits as inputs. For a multiclass system, categorical cross-entropy or negative log-likelihood with logits is used. The logits are preferred due to increased numerical stability.

In the case of binary classification, it is still possible to use a cross-entropy, interpreting each output as the probability of each class. In this case it is preferrable to use a softmax function to normalize the outputs (so they sum to 1) instead of a logistic sigmoid.

In [ ]:
# Binary Cross-entropy Implementation
logits = torch.tensor([0.8])
probs = torch.sigmoid(logits)
target = torch.tensor([1.0])
bce_loss_fn = nn.BCELoss()
bce_logits_loss_fn = nn.BCEWithLogitsLoss()
print(f'BCE (with Probs): {bce_loss_fn(probs, target):.4f}')
print(f'BCE (with Logits): {bce_logits_loss_fn(logits, target):.4f}')

BCE (with Probs): 0.3711
BCE (with Logits): 0.3711


In [ ]:
# Categorical Cross-entropy Implementation
logits = torch.tensor([[1.5, 0.8, 2.1]])
# Note that cce_loss expects log probabilities not raw probabilities
log_probs = torch.log_softmax(logits, dim=1)
target = torch.tensor([2])
cce_loss_fn = nn.NLLLoss()
cce_logits_loss_fn = nn.CrossEntropyLoss()
print(f'CCE (with Probs): {cce_loss_fn(log_probs, target):.4f}')
print(f'CCE (with Logits): {cce_logits_loss_fn(logits, target):.4f}')

CCE (with Probs): 0.5996
CCE (with Logits): 0.5996
